In [1]:
# import libraries
import numpy as np
import pickle
import os
import tqdm
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Conv3D,MaxPooling3D
from keras.optimizers import SGD
from keras import backend as K
from keras.models import load_model

C:\Users\Shaan Desai\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Core dataframe processing functions

In [2]:
def resize_data(x_train,y_train):
    """
        set all input charge densities to have same dimension.. Trim down to z=320
    """
    N = x_train.shape[0]
    new_xtrain = []
    new_ytrain = []
    #Nx = x_train.shape[1]
    #Ny = x_train.shape[2]
    zsize = 200
    #print(x_train[0][0,0,0,0])
    #x_train_M = np.empty((N,Nx,Ny,zsize))
    for i in range(N):
        #condition to check for empty matrices
        if x_train[i].any():
            new_xtrain.append(x_train[i][0,:,:,:zsize])
            new_ytrain.append(y_train[i])
    return new_xtrain,new_ytrain

def train_test(df_charge,train_size):
    """ 
        Get training and test data 
        * Train_size input is fraction of total data size
    """   
    
    msk = np.random.rand(len(df_charge)) < train_size
    charge_data = df_charge['charge_data'].values
    mag_data = df_charge['charge_class'].values
    x_train = charge_data[msk].copy()
    y_train = mag_data[msk].copy()
    x_test = charge_data[~msk].copy()
    y_test = mag_data[~msk].copy()
    x_train,y_train = resize_data(x_train,y_train)
    x_test,y_test = resize_data(x_test,y_test)
    return x_train, y_train, x_test, y_test

#create channels for the image
#only run this once, otherwise you'll keep appending new values at the end
def channels(x_t):
    """
    create channels for the image - currently creates 2 channels (positive and negative)
    only run this once, otherwise you'll keep appending new values at the end
    """
    newvec = []
    for i in range(x_t.shape[0]):
        xt1 = np.copy(x_t[i])
        xt2 = np.copy(x_t[i])
        xt1[xt1<0] = 0
        xt2[xt2>0] = 0
        old_shape = list(xt1.shape)
        old_shape.append(2)
        new_shape = tuple(old_shape)
        newtemp = np.zeros(new_shape)
        newtemp[:,:,:,0] = xt1
        newtemp[:,:,:,1] = xt2
        newvec.append(newtemp)
    return newvec
    

In [3]:
#read input from splitter
df_charge=pd.read_pickle('chgdf_new_input')

In [5]:
#train test split
train_size = 0.80
x_train, y_train, x_test, y_test = train_test(df_charge,train_size)

In [6]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [8]:

#replicate the charge matrix 
#dimx and dimy define the scale at which you want to replicate thus 1,1 means return input
#dimx and dimy need to be integers!
def matmul(mat,dimx,dimy):
    """
    replicate the charge matrix in the x and y directions by scaling factor of dimx,dimy
    dimx,dimy need to be integers
    """
    xrep = np.shape(mat)[0]
    yrep = np.shape(mat)[1]
    zrep = np.shape(mat)[2]
    #assumes only 56 and 60 base dimensions
    if(yrep==56):
        temp = np.zeros(((dimx+1)*xrep,(dimx+1)*yrep,1*zrep))
        #iterate for the integer multiple
        for i in range(int(dimx)):
            for j in range(int(dimy)):
                temp[i*xrep:(i+1)*xrep,yrep*j:(j+1)*yrep,:] = mat
        ret_mat = temp[:(dimx*60),:(dimx*60),:]
    else:
        temp = np.zeros(((dimx)*xrep,(dimx)*yrep,1*zrep))
        #iterate for the integer multiple
        for i in range(int(dimx)):
            for j in range(int(dimy)):
                temp[i*xrep:(i+1)*xrep,yrep*j:(j+1)*yrep,:] = mat
        ret_mat = temp
    return ret_mat        

new_xtrain = []
new_xtest = []
for i in range(x_train.shape[0]):
    new_xtrain.append(matmul(x_train[i],2,2))
for i in range(x_test.shape[0]):
    new_xtest.append(matmul(x_test[i],2,2))
x_train = np.array(new_xtrain)
x_test = np.array(new_xtest)

#print(np.shape(x_train))

In [9]:
#call channels which splits this into a positive and negative sparse matrix
num_classes = 2

x_test = channels(x_test)
x_train = channels(x_train)

# convert class vectors to binary class matrices
# keras likes one hot encoding instead of class names
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)
print(y_train.shape)

(45, 2)
(186, 2)


In [10]:
x_test = np.array(x_test)
x_train = np.array(x_train)

In [11]:
x_test.shape

(45, 120, 120, 200, 2)

In [12]:
# create an empty network model
model = Sequential()

# --- input layer ---
#no padding #width,height,depth,channels
model.add(Conv3D(1, kernel_size=(60,60,60),strides=(1,1,1), activation='relu', input_shape=(120,120,200,2)))
# --- max pool ---
model.add(MaxPooling3D(pool_size=(2,2,2)))

# # --- next layer ---
# # we could double the number of filters as max pool made the 
# # feature maps much smaller 
# # just not doing this to improve runtime
model.add(Conv3D(3, kernel_size=(15,15,15), activation='relu'))
# # --- max pool ---
model.add(MaxPooling3D(pool_size=(2,2,2)))

# # flatten for fully connected classification layer
model.add(Flatten())
# # note that the 2 is the number of classes we have
# # the classes are mutually exclusive so softmax is a good choice
# # --- fully connected layer ---
#model.add(Dense(64, activation='relu'))

# # --- classification ---
model.add(Dense(2, activation='softmax'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 61, 61, 141, 1)    432001    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 30, 70, 1)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 16, 16, 56, 3)     10128     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 8, 8, 28, 3)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5376)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10754     
Total params: 452,883
Trainable params: 452,883
Non-trainable params: 0
_________________________________________________________________


In [13]:
# this does all necessary compiling. In tensorflow this is much quicker than in theano
# the setup is our basic categorical crossentropy with stochastic gradient decent
# we also specify that we want to evaluate our model in terms of accuracy
sgd = SGD(lr=1, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# model.compile(loss='mean_squared_error',
#               optimizer='adam')

In [ ]:
# this is now the actual training
# in addition to the training data we provide validation data
# this data is used to calculate the performance of the model over all the epochs
# this is useful to determine when training should stop
# in our case we just use it to monitor the evolution of the model over the training epochs
# if we use the validation data to determine when to stop the training or which model to save, we 
# should not use the test data, but a separate validation set. 
batch_size = 20

model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=1,
            verbose=1,
            validation_data=(x_test, y_test))

# # once training is complete, let's see how well we have done
# score = model.evaluate(x_test, y_test, verbose=0)


Train on 186 samples, validate on 45 samples
Epoch 1/1


In [34]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.694856584072113
Test accuracy: 0.42105263471603394


In [16]:
model.save('cnn_test1.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model
